____
__Universidad Tecnológica Nacional, Buenos Aires__\
__Ingeniería Industrial__\
__Docente: Martin Palazzo__\
__Ciencia de Datos - Curso I5521 - Turno Jueves Noche__
____

In [ ]:
# importamos las librerías necesarias para trabajar.
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA, KernelPCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Importamos dataset de Wisconsin Breast Cancer

In [ ]:

data = load_breast_cancer()
x = data.data
y = data.target

In [ ]:
type(x)

In [ ]:
np.shape(x)

In [ ]:
type(y)

In [ ]:
np.shape(y)

In [ ]:
sns.heatmap(x)
plt.show()

In [ ]:
sns.heatmap(StandardScaler().fit_transform(x))
plt.show()

# Separamos nuestros datos en Train y Test

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3, random_state=4)

# Realizamos Auto Scaling

In [ ]:
# Step 4: auto scaling train- set (mean = 0, std = 1)

scaler = StandardScaler()
scaler.fit(xtrain)

In [ ]:
xtrain_scal = scaler.transform(xtrain)  
xtest_scal = scaler.transform(xtest)  

In [ ]:
xtrain_scal.shape

In [ ]:
sns.heatmap(xtrain_scal)

In [ ]:
sns.heatmap(xtest_scal)

# Analisis de Componentes Principales (PCA)
Vamos a obtener primeros 10 autovalores y autovectores (eigenvalues y eigenvectors) de la matriz de train "xtrain_scal". 

In [ ]:
# cantidad de componentes a extraer
n_comps = 10
# definimos PCA
pca = PCA(n_components= n_comps)
# fit_transform del PCA a nuestros datos
xpca = pd.DataFrame(pca.fit_transform(xtrain_scal))
# obtenemos los auto-valores
eigenvalues = pca.explained_variance_ratio_

In [ ]:
pca.explained_variance_

In [ ]:
pca.explained_variance_ratio_

In [ ]:
# observamos el shape de la matriz obtenida del PCA. Ver que nuestro dataset se redujo de 30 a 10 dimensiones
xpca.shape

In [ ]:
sns.heatmap(xpca)

# Observamos los autovalores obtenidos del PCA
Recordemos que los autovalores significa la variabilidad que "atrapa" cada componente principal. Obviamente siempre vamos a priorizar las componentes principales que mayor variabilidad capten de los datos de entrenamiento.

In [ ]:
components = range(1,n_comps + 1)
plt.bar(components,eigenvalues)
plt.xticks(components)
plt.title('Explained variance of top ' + str(n_comps) + ' principal components')
plt.xlabel('Top ' +str(n_comps) + ' Principal Components')
plt.ylabel('Explained Variance')
plt.show()

Ahora graficaremos la variabilidad explicada acumulada

In [ ]:
fig, ax1 = plt.subplots()

components = range(1, n_comps + 1)

# Graficar las barras de varianza explicada en el eje izquierdo
ax1.bar(components, eigenvalues, label='Explained Variance', color='blue')
ax1.set_xlabel('Top ' + str(n_comps) + ' Principal Components')
ax1.set_ylabel('Explained Variance', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# Crear un segundo eje para la varianza acumulada
ax2 = ax1.twinx()
cumulative_variance = np.cumsum(eigenvalues) / np.sum(eigenvalues)

# Graficar la varianza acumulada en el eje derecho
ax2.plot(components, cumulative_variance, color='red', marker='o', label='Cumulative Variance')
ax2.set_ylabel('Cumulative Variance', color='red')
ax2.tick_params(axis='y', labelcolor='red')

# Configurar el título y las leyendas
plt.title('Explained variance of top ' + str(n_comps) + ' principal components')

fig.tight_layout()  # Ajustar el layout para que no se superpongan los ejes
plt.show()


# Scatter plot del PCA
Con las 2 primeras componentes principales **antes** de la selección de variables visualizamos nuestras muestras/samples de train. **Atencion!** vean que estamos visualizando datos de 30 dimensiones en solamente 2 :). Otro punto importante es que con las dos primeras componentes principales captamos el 65 % aprox de la variabilidad de mis datos (hacer explained variance de PC1 + PC2)

In [ ]:
sns.set_context("talk", font_scale=0.7)
plt.figure(figsize=(9,6))
plt.scatter(xpca.loc[(ytrain == 0).ravel(),0],xpca.loc[(ytrain == 0).ravel(),1], alpha = 0.3, label = 'B')
plt.scatter(xpca.loc[(ytrain == 1).ravel(),0],xpca.loc[(ytrain == 1).ravel(),1], alpha = 0.3, label = 'M')
plt.xlabel('Principal Component 1 , Explainded Variance = ' + str(np.round(eigenvalues[0],2)))
plt.ylabel('Principal Component 2 , Explainded Variance = ' + str(np.round(eigenvalues[1],2)))
plt.title('Principal Component Analysis before feature selection')
plt.legend(loc='upper right')
plt.show()

Estamos visualizando datos de alta dimension en 2 dimensiones :):):):). Ojo que con las dos primeras componentes visualizamos el 63% de la variabilidad del dataset original. Es decir que sirve para darnos una idea aunque lo que se aconseja es quedarse con las componentes que acumulan aunque sea el 80% de la variabilidad original.

In [ ]:
# vamos a visualizar en 3d
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xpca.iloc[(ytrain == 0).ravel(),0], xpca.iloc[(ytrain == 0).ravel(),1], xpca.iloc[(ytrain == 0).ravel(),2])
ax.scatter(xpca.iloc[(ytrain == 1).ravel(),0], xpca.iloc[(ytrain == 1).ravel(),1], xpca.iloc[(ytrain == 1).ravel(),2])
ax.set_xlabel('P. Comp. 1')
ax.set_ylabel('P. Comp. 2')
ax.set_zlabel('P. Comp. 3')
plt.show()

# kernel-PCA

Computamos el desvio standard de cada feature. Luego decidimos quedarnos con aquellas cuyo STDEV sea mayor a un cuantil determinado. Eso resultara de un vector "mascara" booleano que nos serirá para filtrar el dataset original. Vean que este metodo es no supervisado aunque su debilidad es que es univariado.

In [ ]:
# cantidad de componentes a extraer
n_comps = 100
# definimos PCA
kpca = KernelPCA(n_components= n_comps, kernel = 'rbf', gamma = 0.05)
# fit_transform del PCA a nuestros datos
xkpca = pd.DataFrame(kpca.fit_transform(xtrain_scal))


In [ ]:
sns.heatmap(xkpca)
plt.show()

In [ ]:
sns.set_context("talk", font_scale=0.7)
plt.figure(figsize=(9,6))
plt.scatter(xkpca.loc[(ytrain == 0).ravel(),0],xkpca.loc[(ytrain == 0).ravel(),1], alpha = 0.7, label = 'B')
plt.scatter(xkpca.loc[(ytrain == 1).ravel(),0],xkpca.loc[(ytrain == 1).ravel(),1], alpha = 0.7, label = 'M')
#plt.xlabel('Principal Component 1 , Explainded Variance = ' + str(np.round(eigenvalues[0],2)))
#plt.ylabel('Principal Component 2 , Explainded Variance = ' + str(np.round(eigenvalues[1],2)))
plt.title('Principal Component Analysis before feature selection')
plt.legend(loc='upper right')
plt.show()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(xkpca.iloc[(ytrain == 0).ravel(),0], xkpca.iloc[(ytrain == 0).ravel(),1], xkpca.iloc[(ytrain == 0).ravel(),2])
ax.scatter(xkpca.iloc[(ytrain == 1).ravel(),0], xkpca.iloc[(ytrain == 1).ravel(),1], xkpca.iloc[(ytrain == 1).ravel(),2])
ax.set_xlabel('P. Comp. 1')
ax.set_ylabel('P. Comp. 2')
ax.set_zlabel('P. Comp. 3')
plt.show()

# Asignacion 01:
Partiendo de los datos originales crear un nuevo train-test split donde train es el 10% de los datos. Reducir la dimension con PCA y kPCA. Clasificar usando las proyecciones obtenidas con logistic regression. Medir la calidad de la clasificacion en area debajo de la curva y accuracy.

In [ ]:
# Imports

####################
# Importar librerias
####################

In [ ]:
# train-test split con 0.9 test size
# guardar el split en xtrain , xtest
# definir scaler .fit(xtrain)
# ...
# obtenemos xtrain_scal2 xtest_scal2


# aplicar PCA y kPCA sobre el xtrain_scal2

# guardar las proyecciones obtenidas en xpca2_train , xpca2_test
# guardar las proyecciones obtenidas en xkpca2_train , xkpca2_test

# pca2 = PCA(n_comps = ....)
# pca2.fit(xtrain_scal2)
# zpca2_train = pca2.transform(xtrain_scal2)
# zpca2_test = pca2.transform(xtest_scal2)

# es decir obtener xkpca2 , xpca2


# definir un clasificador logistic regression tanto para entrenarse en la proyeccion del PCA y otro para la del kPCA

# lr_pca = LogisticRegression(....blbalabla)
# lr_kpca = LogisticRegression(....blbalabla)

# lr_pca.fit(zpca2_train)
# lr_kpca.fit(zkpca2_train)


# una vez que tengo entrenado (fiteado) el modelo -> 
# ypred_pca = lr_pca.predict(zpca2_test)
# ypred_kpca = lr_kpca.predict(zkpca2_test)


# acc_lr_pca =  accuracy_score(ypred_pca, ytest)
# acc_lr_kpca =  accuracy_score(ypred_kpca, ytest)